In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
import os

# Define the path to the OneDrive folder
onedrive_path = os.path.expanduser('/Users/ankitkumar/Library/CloudStorage/OneDrive-GeorgeMasonUniversity-O365Production/HackFax/')

# Define the path to the specific file within the OneDrive folder
file_path = os.path.join(onedrive_path, 'samplegeocodiooutput_acseconomics.csv')

# Read the data into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()